In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

This cell takes all of the input parameters.

In [2]:
nh = 8
refRatio = 2
CFL = 0.5
deriv = 'U'
order = 1

This cell contains the relevant switch information.

In [3]:
if (deriv == 'U'):
    DiffFunc = TT.Upwind #ST.Upwind
else:
    DiffFunc = TT.CenterDiff #ST.CenterDiff

This cell instantiates the grid.

In [4]:
omega = BT.Grid(nh)
finehalf = list(np.arange(int(nh / refRatio)))# + int(nh / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_min = omega.nh_min
nh_max = omega.nh_max

This cell runs a test on the derivative operator on a polynomial which matches the given order.

In [5]:
TT.DerivPolyTest(omega, DiffFunc, order)

x:
[0.03125 0.09375 0.15625 0.21875 0.28125 0.34375 0.40625 0.46875 0.5625  0.6875  0.8125  0.9375 ]

Polynomial Function:
p(x) =   
1 x + 1
p(x) =
 [1.03125 1.09375 1.15625 1.21875 1.28125 1.34375 1.40625 1.46875 1.5625  1.6875  1.8125  1.9375 ]

Polynomial Derivative:
dp(x)/dx =  
1
Theoretical:
dp(x)/dx =
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Actual:
dp(x)/dx =
 [-9.66666667  1.          1.          1.          1.          1.          1.          1.          1.          1.          1.          1.        ]

